# MGT 858: Database Systems
## Hospital Data Schema

The code written below seeks to add data to a postgresql database

### Import packages

In [91]:
import random
import numpy as np
import pandas as pd
from datetime import timedelta, datetime
import datetime

### Add patients 

In [161]:
# Lists of first names, last names, and addresses
first_names = ['Michael', 'Christopher', 'Jessica', 'Matthew', 'Ashley', 'Jennifer', 
               'Joshua', 'Amanda', 'Daniel', 'David', 'James', 'Robert', 'John', 'Joseph', 
               'Andrew', 'Ryan', 'Brandon', 'Jason', 'Justin', 'Sarah', 'William', 'Jonathan', 
               'Stephanie', 'Brian', 'Nicole', 'Nicholas', 'Anthony', 'Heather', 'Eric', 'Elizabeth', 
               'Adam', 'Megan', 'Melissa', 'Kevin', 'Steven', 'Thomas', 'Timothy', 'Christina', 'Kyle', 
               'Rachel', 'Laura', 'Lauren', 'Amber', 'Brittany', 'Danielle', 'Richard', 'Kimberly', 'Jeffrey', 
               'Amy', 'Crystal', 'Michelle', 'Tiffany', 'Jeremy', 'Benjamin', 'Mark', 'Emily', 'Aaron', 'Charles', 
               'Rebecca', 'Jacob', 'Stephen', 'Patrick', 'Sean', 'Erin', 'Zachary', 'Jamie', 'Kelly', 'Samantha', 
               'Nathan', 'Sara', 'Dustin', 'Paul', 'Angela', 'Tyler', 'Scott', 'Katherine', 'Andrea', 'Gregory', 
               'Erica', 'Mary', 'Travis', 'Lisa', 'Kenneth', 'Bryan', 'Lindsey', 'Kristen', 'Jose', 'Alexander', 
               'Jesse', 'Katie', 'Lindsay', 'Shannon', 'Vanessa', 'Courtney', 'Christine', 'Alicia', 'Cody', 
               'Allison', 'Bradley', 'Samuel', 'Shawn', 'April', 'Derek', 'Kathryn', 'Kristin', 'Chad', 'Jenna', 
               'Tara', 'Maria', 'Krystal', 'Jared', 'Anna', 'Edward', 'Julie', 'Peter', 'Holly', 'Marcus', 
               'Kristina', 'Natalie', 'Jordan', 'Victoria', 'Jacqueline', 'Corey', 'Keith', 'Monica', 'Juan', 
               'Donald', 'Cassandra','Meghan', 'Joel', 'Shane', 'Phillip', 'Patricia', 'Brett', 'Ronald', 
               'Catherine', 'George', 'Antonio', 'Cynthia', 'Stacy', 'Kathleen', 'Raymond', 'Carlos', 'Brandi', 
               'Douglas', 'Nathaniel', 'Ian', 'Craig', 'Brandy', 'Alex', 'Valerie', 'Veronica', 'Cory', 'Whitney', 
               'Gary', 'Derrick', 'Philip', 'Luis', 'Diana', 'Chelsea', 'Leslie', 'Caitlin', 'Leah', 'Natasha', 
               'Erika', 'Casey', 'Latoya', 'Erik', 'Dana', 'Victor', 'Brent']

last_names = ['Smith', 'Johnson', 'Williams', 'Jones', 'Brown', 'Davis', 'Miller', 'Wilson', 'Moore', 'Taylor', 
               'Anderson', 'Thomas', 'Jackson', 'White', 'Harris', 'Martin', 'Thompson', 'Garcia', 'Martinez', 
               'Robinson', 'Clark', 'Rodriguez', 'Lewis', 'Lee', 'Walker', 'Hall', 'Allen', 'Young', 'Hernandez', 
               'King', 'Wright', 'Lopez', 'Hill', 'Scott', 'Green', 'Adams', 'Baker', 'Gonzalez', 'Nelson', 'Carter', 
               'Mitchell', 'Perez', 'Roberts', 'Turner', 'Phillips', 'Campbell', 'Parker', 'Evans', 'Edwards', 
               'Collins', 'Stewart', 'Sanchez', 'Morris', 'Rogers', 'Reed', 'Cook', 'Morgan', 'Bell', 'Murphy', 
               'Bailey', 'Rivera', 'Cooper', 'Richardson', 'Cox', 'Howard', 'Ward', 'Torres', 'Peterson', 'Gray', 
               'Ramirez', 'James', 'Watson', 'Brooks', 'Kelly', 'Sanders', 'Price', 'Bennett', 'Wood', 'Barnes', 
               'Ross', 'Henderson', 'Coleman', 'Jenkins', 'Perry', 'Powell', 'Long', 'Patterson', 'Hughes', 'Flores', 
               'Washington', 'Butler', 'Simmons', 'Foster', 'Gonzales', 'Bryant', 'Alexander', 'Russell', 'Griffin ', 
               'Diaz', 'Hayes']

addresses = ['Oakmont Way', 'Glenndale Blvd','Sherwood Ln','Calle Acanta','Chapel St','Whitney Avenue','Dwight Rd',
             'Howe St','Burrows Rd','Great King Crossing','Westeros Way','Kings Landing','Yale Blvd','Handsome Dan St',
            'Wall Street','Abbey Road']

# Pre-existing SSNs for patients
SSN_list = [500000001, 500000002, 500000003, 500000004, 500000005, 500000006, 500000007, 500000008]

# Defines a function to add patients and print INSERT INTO statements
# num_to_add = the number of new randomly generated patients you would like to add
def add_patients(num_to_add):
    for i in range(num_to_add):
        SSN = 500000009 + i
        SSN_list.append(SSN)
        full_name = random.choice(first_names) + " " + random.choice(last_names)
        address = str(random.randint(100,999)) + " " + random.choice(addresses)
        phone_number = str(random.randint(100,999)) + "-" + str(random.randint(1000,9999))
        insurance_id = random.randint(10000000,99999999)
        pcp = random.randint(1,13)
        print(f"INSERT INTO patients (SSN, patient_name, address, phone, insurance_id, primary_care_provider) VALUES({SSN}, '{full_name}', '{address}', '{phone_number}', {insurance_id}, {pcp});")
        
add_patients(100)
    

INSERT INTO patients (SSN, patient_name, address, phone, insurance_id, primary_care_provider) VALUES(500000009, 'Jose Richardson', '731 Sherwood Ln', '541-2418', 53126058, 13);
INSERT INTO patients (SSN, patient_name, address, phone, insurance_id, primary_care_provider) VALUES(500000010, 'Amy Mitchell', '888 Kings Landing', '272-8472', 65686429, 2);
INSERT INTO patients (SSN, patient_name, address, phone, insurance_id, primary_care_provider) VALUES(500000011, 'Kyle Edwards', '234 Wall Street', '855-9218', 31823330, 11);
INSERT INTO patients (SSN, patient_name, address, phone, insurance_id, primary_care_provider) VALUES(500000012, 'Anthony Jackson', '328 Oakmont Way', '949-1540', 16534668, 6);
INSERT INTO patients (SSN, patient_name, address, phone, insurance_id, primary_care_provider) VALUES(500000013, 'Peter Sanchez', '113 Abbey Road', '902-7151', 53425912, 12);
INSERT INTO patients (SSN, patient_name, address, phone, insurance_id, primary_care_provider) VALUES(500000014, 'Amber Reed'

## Add patient stays

In [162]:
# Define hospital rooms
rooms = ["101","102","103","104","105","106","107","108","109","110",
        "201","202","203","204","205","206","207","208","209","210",
        "301","302","303","304","305","306","307","308","309","310",
        "401","402","403","404","405","406","407","408","409","410",
        "501","502","503","504","505","506","507","508","509","510",
        "601","602","603","604","605","606","607","608","609","610"]

# Defines dates for which we will generate hospital stays
dates = pd.to_datetime(pd.date_range(start="2020-01-01",end="2021-03-24"))

# Defines a function that returns a dataframe of randomly generated patient stays
def patient_stays():
    cols = ['date','patient_SSN','room_number','stay_start','stay_end']
    data = pd.DataFrame(columns = cols)
    for date in dates:
        daily_arrival = random.randint(0,5) # Here, we say on any day, between 0-5 people will arrive randomly
        daily_patient_arrival = random.sample(SSN_list,daily_arrival)
        for patient in daily_patient_arrival:
            stay_length = np.random.normal(4,1) # Here, we say people stay on average 4 days with a 1 day std.dev.
            stay_start = datetime.datetime.strptime(str(date),'%Y-%m-%d %H:%M:%S').date()
            stay_end = stay_start + datetime.timedelta(days=stay_length)
            room = random.sample(rooms,1)
            assigned_room = " ".join(str(x) for x in room)
            data = data.append({'date':stay_start,'patient_SSN':patient,'room_number':assigned_room,'stay_start':stay_start,'stay_end':stay_end},ignore_index=True)

    return data

## Check added patient stay data for duplicates or error entries 

In [163]:
data = patient_stays()

data = data.drop_duplicates(subset=['date','patient_SSN'])
clean_data = data.drop_duplicates(subset=['date','room_number'])

## Iterate over cleaned data frame and write insert statements

In [164]:
for ind in clean_data.index:
    print(f"INSERT INTO stays (patient, room, stay_duration) VALUES({clean_data['patient_SSN'][ind]},{clean_data['room_number'][ind]},daterange('{clean_data['stay_start'][ind]}','{clean_data['stay_end'][ind]}','[]'));")

INSERT INTO stays (patient, room, stay_duration) VALUES(500000076,405,daterange('2020-01-01','2020-01-05','[]'));
INSERT INTO stays (patient, room, stay_duration) VALUES(500000042,207,daterange('2020-01-02','2020-01-07','[]'));
INSERT INTO stays (patient, room, stay_duration) VALUES(500000030,606,daterange('2020-01-02','2020-01-06','[]'));
INSERT INTO stays (patient, room, stay_duration) VALUES(500000051,310,daterange('2020-01-02','2020-01-05','[]'));
INSERT INTO stays (patient, room, stay_duration) VALUES(500000102,404,daterange('2020-01-02','2020-01-04','[]'));
INSERT INTO stays (patient, room, stay_duration) VALUES(500000012,506,daterange('2020-01-03','2020-01-05','[]'));
INSERT INTO stays (patient, room, stay_duration) VALUES(500000081,507,daterange('2020-01-03','2020-01-05','[]'));
INSERT INTO stays (patient, room, stay_duration) VALUES(500000017,403,daterange('2020-01-03','2020-01-07','[]'));
INSERT INTO stays (patient, room, stay_duration) VALUES(500000056,205,daterange('2020-01

INSERT INTO stays (patient, room, stay_duration) VALUES(500000017,505,daterange('2021-01-10','2021-01-14','[]'));
INSERT INTO stays (patient, room, stay_duration) VALUES(500000081,304,daterange('2021-01-10','2021-01-12','[]'));
INSERT INTO stays (patient, room, stay_duration) VALUES(500000102,307,daterange('2021-01-10','2021-01-14','[]'));
INSERT INTO stays (patient, room, stay_duration) VALUES(500000099,508,daterange('2021-01-10','2021-01-12','[]'));
INSERT INTO stays (patient, room, stay_duration) VALUES(500000010,304,daterange('2021-01-11','2021-01-14','[]'));
INSERT INTO stays (patient, room, stay_duration) VALUES(500000099,201,daterange('2021-01-12','2021-01-17','[]'));
INSERT INTO stays (patient, room, stay_duration) VALUES(500000015,108,daterange('2021-01-12','2021-01-16','[]'));
INSERT INTO stays (patient, room, stay_duration) VALUES(500000084,106,daterange('2021-01-13','2021-01-16','[]'));
INSERT INTO stays (patient, room, stay_duration) VALUES(500000075,506,daterange('2021-01